In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Reading The Data from CSV

In [2]:
serra_df = pd.read_csv('../Data/sierraleone-bumbuna.csv')
serra_df.head(10)

,Timestamp,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
0,2021-10-30 00:01,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.1,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN
1,2021-10-30 00:02,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.2,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN
2,2021-10-30 00:03,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.2,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN
3,2021-10-30 00:04,-0.7,0.0,-0.8,0.0,0.0,21.9,99.3,0.0,0.0,0.0,0.0,0.0,1002,0,0.1,22.3,22.6,NaN
4,2021-10-30 00:05,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.3,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN
5,2021-10-30 00:06,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.3,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.7,NaN
6,2021-10-30 00:07,-0.7,0.0,-0.8,0.0,0.0,22.0,99.4,0.1,0.7,0.2,240.4,0.0,1002,0,0.0,22.3,22.7,NaN
7,2021-10-30 00:08,-0.7,0.0,-0.8,0.0,0.0,22.0,99.5,0.2,0.7,0.4,233.5,1.6,1002,0,0.0,22.4,22.7,NaN
8,2021-10-30 00:09,-0.7,-0.1,-0.8,0.0,0.0,22.0,99.5,0.7,1.1,0.5,232.0,0.3,1002,0,0.0,22.4,22.7,NaN
9,2021-10-30 00:10,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.2,0.6,1.1,0.5,232.5,0.3,1002,0,0.0,22.4,22.7,NaN


In [3]:
serra_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525600 entries, 0 to 525599
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Timestamp      525600 non-null  object 
 1   GHI            525600 non-null  float64
 2   DNI            525600 non-null  float64
 3   DHI            525600 non-null  float64
 4   ModA           525600 non-null  float64
 5   ModB           525600 non-null  float64
 6   Tamb           525600 non-null  float64
 7   RH             525600 non-null  float64
 8   WS             525600 non-null  float64
 9   WSgust         525600 non-null  float64
 10  WSstdev        525600 non-null  float64
 11  WD             525600 non-null  float64
 12  WDstdev        525600 non-null  float64
 13  BP             525600 non-null  int64  
 14  Cleaning       525600 non-null  int64  
 15  Precipitation  525600 non-null  float64
 16  TModA          525600 non-null  float64
 17  TModB          525600 non-nul

cheaking missing values

In [4]:
serra_df.isnull().sum()

Timestamp             0
GHI                   0
DNI                   0
DHI                   0
ModA                  0
ModB                  0
Tamb                  0
RH                    0
WS                    0
WSgust                0
WSstdev               0
WD                    0
WDstdev               0
BP                    0
Cleaning              0
Precipitation         0
TModA                 0
TModB                 0
Comments         525600
dtype: int64

Checking for wrong entries like symbols -,?,#,*,etc

In [5]:
for col in serra_df.columns:
    print('{} : {}'.format(col,serra_df[col].unique()))

Timestamp : ['2021-10-30 00:01' '2021-10-30 00:02' '2021-10-30 00:03' ...
 '2022-10-29 23:58' '2022-10-29 23:59' '2022-10-30 00:00']
GHI : [-7.000e-01 -6.000e-01 -8.000e-01 ...  1.381e+03  1.394e+03  1.329e+03]
DNI : [-1.00e-01  0.00e+00 -2.00e-01 ...  9.29e+02 -5.20e+00  9.05e+02]
DHI : [ -0.8  -0.9  -1.  ... 622.5 675.6 682.1]
ModA : [0.000e+00 1.000e-01 2.000e-01 ... 1.294e+03 1.214e+03 1.305e+03]
ModB : [0.000e+00 1.000e-01 2.000e-01 ... 1.354e+03 1.339e+03 1.238e+03]
Tamb : [21.9 22.  22.1 22.2 22.3 22.4 22.5 22.6 22.7 22.8 22.9 23.  23.1 23.2
 23.3 23.4 23.5 23.6 23.7 23.8 23.9 24.  24.1 24.2 24.3 24.4 24.5 24.6
 24.7 24.8 24.9 25.  25.1 25.2 25.3 25.4 25.6 25.5 25.7 25.8 25.9 26.
 26.1 26.2 26.4 26.5 26.3 26.6 26.8 26.9 27.  27.1 27.2 27.4 27.5 27.6
 27.7 27.8 27.9 28.  28.1 28.2 28.5 28.7 28.4 28.3 28.6 28.8 29.  29.2
 29.1 29.4 29.6 29.5 29.3 28.9 29.7 30.  30.1 29.9 29.8 30.2 30.3 30.4
 27.3 26.7 21.8 21.7 21.6 30.5 30.7 31.  31.3 31.5 31.4 31.2 31.1 30.9
 30.8 30.6 31.6 31.7